In [8]:
import pandas as pd
import numpy as np
%matplotlib
import matplotlib.pyplot as plt
import seaborn as sn

Using matplotlib backend: TkAgg


In [9]:
df = pd.read_csv("predictive_maintenance.csv")
df

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure
...,...,...,...,...,...,...,...,...,...,...
9995,9996,M24855,M,298.8,308.4,1604,29.5,14,0,No Failure
9996,9997,H39410,H,298.9,308.4,1632,31.8,17,0,No Failure
9997,9998,M24857,M,299.0,308.6,1645,33.4,22,0,No Failure
9998,9999,H39412,H,299.0,308.7,1408,48.5,25,0,No Failure


<h2> Um pouco sobre os dados </h2>

1).***UDI*** (Identificador Único): Este é um identificador exclusivo para cada ponto de dados, o que sugere que cada ponto de dados corresponde a um evento ou observação específica em um processo de fabricação. Cada UID pode estar associado a uma operação ou evento específico da máquina.
    
2).***ProductID***: Este recurso representa as variantes de qualidade do produto (baixa, média ou alta) e inclui um número de série específico da variante. Isto pode indicar diferentes tipos de produtos ou variações produzidas pelas máquinas.
    
3).***Temperatura do ar [K]***: Este recurso representa a temperatura do ar em Kelvin, que pode ser um parâmetro relevante para o processo de fabricação, indicando potencialmente as condições ambientais ou configurações das máquinas.
    
4).***Temperatura do processo [K]***: Este recurso está relacionado à temperatura do processo em Kelvin, que é gerada usando um processo de passeio aleatório e normalizada. Pode ser um parâmetro crítico no processo de fabricação, afetando a qualidade do produto e a operação da máquina.
    
5).***Velocidade de rotação [rpm]***: Este recurso representa a velocidade de rotação das máquinas, que é calculada com base na potência e inclui ruído normalmente distribuído. É um parâmetro essencial para a compreensão do funcionamento da máquina.
    
6).***Torque [Nm]***: Os valores de torque são normalmente distribuídos em torno de 40 Nm com características específicas. Este recurso provavelmente indica o torque aplicado pelas máquinas durante o processo de fabricação.
    
7).***Desgaste da ferramenta [min]***: A característica de desgaste da ferramenta está associada às variantes de qualidade (H/M/L) e adiciona minutos de desgaste da ferramenta à ferramenta utilizada no processo. Sugere que as ferramentas sejam utilizadas em conjunto com as máquinas durante o processo de fabricação.
    
8). ***Rótulo 'falha da máquina'***: Este rótulo indica se a máquina falhou em um determinado ponto de dados para qualquer um dos modos de falha especificados. Isto sugere que o conjunto de dados inclui informações sobre falhas de máquinas, e o conjunto de dados pode incluir diferentes tipos de máquinas com diferentes modos de falha.

<h2>A falha da máquina consiste em cinco modos de falha independentes</h2>

falha por desgaste da ferramenta (TWF): a ferramenta será substituída ou falha em um tempo de desgaste da ferramenta selecionado aleatoriamente entre 200 - 240 minutos (120 vezes em nosso conjunto de dados). Neste momento, a ferramenta foi substituída 69 vezes e falhou 51 vezes (atribuída aleatoriamente).

falha de dissipação de calor (HDF): a dissipação de calor causa uma falha no processo, se a diferença entre a temperatura do ar e do processo for inferior a 8,6 K e a velocidade de rotação da ferramenta for inferior a 1380 rpm. Este é o caso de 115 pontos de dados.

falha de energia (PWF): o produto do torque e da velocidade de rotação (em rad/s) é igual à potência necessária para o processo. Se esta potência estiver abaixo de 3.500 W ou acima de 9.000 W, o processo falha, o que ocorre 95 vezes em nosso conjunto de dados.

falha por sobretensão (OSF): se o produto do desgaste da ferramenta e o torque exceder 11.000 minNm para a variante do produto L (12.000 M, 13.000 H), o processo falha devido à sobretensão. Isso é verdade para 98 pontos de dados.

falhas aleatórias (RNF): cada processo tem 0,1% de chance de falhar, independentemente de seus parâmetros de processo. Este é o caso de apenas 5 pontos de dados, menos do que poderia ser esperado para 10.000 pontos de dados no nosso conjunto de dados.

Se pelo menos um dos modos de falha acima for verdadeiro, o processo falha e o rótulo de 'falha da máquina' é definido como 1. Portanto, não é transparente para o método de aprendizado de máquina qual dos modos de falha causou a falha do processo

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   UDI                      10000 non-null  int64  
 1   Product ID               10000 non-null  object 
 2   Type                     10000 non-null  object 
 3   Air temperature [K]      10000 non-null  float64
 4   Process temperature [K]  10000 non-null  float64
 5   Rotational speed [rpm]   10000 non-null  int64  
 6   Torque [Nm]              10000 non-null  float64
 7   Tool wear [min]          10000 non-null  int64  
 8   Target                   10000 non-null  int64  
 9   Failure Type             10000 non-null  object 
dtypes: float64(3), int64(4), object(3)
memory usage: 781.4+ KB


In [11]:
df["Target"].value_counts()

Target
0    9661
1     339
Name: count, dtype: int64

In [12]:
df["Failure Type"].value_counts()

Failure Type
No Failure                  9652
Heat Dissipation Failure     112
Power Failure                 95
Overstrain Failure            78
Tool Wear Failure             45
Random Failures               18
Name: count, dtype: int64

In [13]:
# Obtém instâncias onde 'Target' é 0, mas o tipo de falha não é "Sem falha"
selected_instances = df[(df['Target'] == 0) & (df['Failure Type'] != 'No Failure')]
selected_instances

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
1221,1222,M16081,M,297.0,308.3,1399,46.4,132,0,Random Failures
1302,1303,L48482,L,298.6,309.8,1505,45.7,144,0,Random Failures
1748,1749,H31162,H,298.4,307.7,1626,31.1,166,0,Random Failures
2072,2073,L49252,L,299.6,309.5,1570,35.5,189,0,Random Failures
2559,2560,L49739,L,299.3,309.0,1447,50.4,140,0,Random Failures
3065,3066,M17925,M,300.1,309.2,1687,27.7,95,0,Random Failures
3452,3453,H32866,H,301.6,310.5,1602,32.3,2,0,Random Failures
5471,5472,L52651,L,302.7,312.3,1346,61.2,170,0,Random Failures
5489,5490,L52669,L,302.6,312.1,1499,35.0,215,0,Random Failures
5495,5496,H34909,H,302.9,312.5,1357,55.0,12,0,Random Failures


In [14]:
# Obtém instâncias onde 'Target' é 1, mas o tipo de falha não é "Sem falha"
selected_instances1 = df[(df['Target'] == 1) & (df['Failure Type'] == 'No Failure')]
selected_instances1

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
1437,1438,H30851,H,298.8,309.9,1439,45.2,40,1,No Failure
2749,2750,M17609,M,299.7,309.2,1685,28.9,179,1,No Failure
4044,4045,M18904,M,301.9,310.9,1419,47.7,20,1,No Failure
4684,4685,M19544,M,303.6,311.8,1421,44.8,101,1,No Failure
5536,5537,M20396,M,302.3,311.8,1363,54.0,119,1,No Failure
5941,5942,L53121,L,300.6,310.7,1438,48.5,78,1,No Failure
6478,6479,L53658,L,300.5,309.8,1663,29.1,145,1,No Failure
8506,8507,L55686,L,298.4,309.6,1710,27.3,163,1,No Failure
9015,9016,L56195,L,297.2,308.1,1431,49.7,210,1,No Failure


In [15]:
# Altere os valores em 'Target' de 0 para 1 onde 'Failure_Type' é 'random'
df.loc[df['Failure Type'] == 'Random Failures', 'Target'] = 1
df.loc[df['Failure Type'] == 'No Failure', 'Target'] = 0

In [16]:
selected_instances1 = df[(df['Target'] == 1) & (df['Failure Type'] == 'No Failure')]
selected_instances1

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type


In [17]:
df.Target.value_counts()

Target
0    9652
1     348
Name: count, dtype: int64

In [18]:
df["Failure Type"].value_counts()

Failure Type
No Failure                  9652
Heat Dissipation Failure     112
Power Failure                 95
Overstrain Failure            78
Tool Wear Failure             45
Random Failures               18
Name: count, dtype: int64

In [19]:
df.hist(figsize=(20,15))

array([[<Axes: title={'center': 'UDI'}>,
        <Axes: title={'center': 'Air temperature [K]'}>,
        <Axes: title={'center': 'Process temperature [K]'}>],
       [<Axes: title={'center': 'Rotational speed [rpm]'}>,
        <Axes: title={'center': 'Torque [Nm]'}>,
        <Axes: title={'center': 'Tool wear [min]'}>],
       [<Axes: title={'center': 'Target'}>, <Axes: >, <Axes: >]],
      dtype=object)

In [20]:
sn.pairplot(df,hue="Target",palette="husl")

In [22]:
##Importando nova amostra do pacote *sklearn.utils*. 
from sklearn.utils import resample

# Separe o caso de 0 e 1
no_failure_0 = df[df.Target==0]
failure_1 = df[df.Target==1]

##Amplie os casos de failuer(1).
df_minority_upsampled = resample(failure_1,replace=True,n_samples=2000)

# Combine a classe majoritária com a classe minoritária ampliada
new_df = pd.concat([no_failure_0, df_minority_upsampled])

In [23]:
from sklearn.utils import shuffle
shuffle_df = shuffle(new_df)

In [24]:
shuffle_df

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
1793,1794,L48973,L,298.1,307.8,1761,28.2,76,0,No Failure
8289,8290,L55469,L,298.9,310.6,1406,43.8,12,0,No Failure
5267,5268,M20127,M,303.5,312.9,1438,42.8,106,0,No Failure
2642,2643,L49822,L,299.8,309.5,1542,40.3,141,0,No Failure
9802,9803,H39216,H,298.4,309.3,1984,19.2,132,0,No Failure
...,...,...,...,...,...,...,...,...,...,...
5275,5276,L52455,L,303.7,313.1,1423,41.2,124,0,No Failure
4618,4619,M19478,M,303.0,311.3,1335,53.6,164,1,Heat Dissipation Failure
2364,2365,L49544,L,299.2,308.5,1366,49.7,74,0,No Failure
2332,2333,L49512,L,299.2,308.5,1378,50.4,220,1,Overstrain Failure


In [25]:
shuffle_df['Failure Type'].value_counts()

Failure Type
No Failure                  9652
Heat Dissipation Failure     653
Power Failure                508
Overstrain Failure           444
Tool Wear Failure            275
Random Failures              120
Name: count, dtype: int64

In [26]:
shuffle_df.Type.value_counts()

Type
L    7173
M    3356
H    1123
Name: count, dtype: int64

In [27]:
sn.scatterplot(data=shuffle_df, x=shuffle_df['Rotational speed [rpm]'],y=shuffle_df['Torque [Nm]'],hue="Failure Type",palette="tab10")

<Axes: xlabel='Rotational speed [rpm]', ylabel='Torque [Nm]'>

In [28]:
subset_df = shuffle_df[shuffle_df["Target"] == 1].reset_index(drop=True)
sn.scatterplot(data=subset_df, x=subset_df['Rotational speed [rpm]'],y=subset_df['Torque [Nm]'],hue="Failure Type",palette="tab10")

<Axes: xlabel='Rotational speed [rpm]', ylabel='Torque [Nm]'>

In [29]:
sn.scatterplot(data=subset_df, x=subset_df['Tool wear [min]'],y=subset_df['Failure Type'],palette="tab10")

C:\Users\rianb\AppData\Local\Temp\ipykernel_9160\1621668466.py:1: UserWarning: Ignoring `palette` because no `hue` variable has been assigned.
  sn.scatterplot(data=subset_df, x=subset_df['Tool wear [min]'],y=subset_df['Failure Type'],palette="tab10")


<Axes: xlabel='Tool wear [min]', ylabel='Failure Type'>

In [30]:
sn.scatterplot(data=subset_df, x=subset_df['Process temperature [K]'],y=subset_df['Failure Type'],palette="tab10")

C:\Users\rianb\AppData\Local\Temp\ipykernel_9160\988469026.py:1: UserWarning: Ignoring `palette` because no `hue` variable has been assigned.
  sn.scatterplot(data=subset_df, x=subset_df['Process temperature [K]'],y=subset_df['Failure Type'],palette="tab10")


<Axes: xlabel='Process temperature [K]', ylabel='Failure Type'>

In [31]:
# Dropei o ID do produto aqui porque, em minha análise, o ID do produto não tem tanta importância
# Tambem há a possibilidade de ter dados com muita cardinalidade, então esqueçamos

encodeded_df = pd.get_dummies(shuffle_df.drop(["Product ID",'Target', 'Failure Type'],axis=1),drop_first=True)


In [32]:
label_1=shuffle_df['Target']
label_2 = shuffle_df["Failure Type"]

In [33]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled = scaler.fit_transform(encodeded_df)

In [34]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(scaled,label_1,test_size=0.2, random_state=42)

<h2>Construção de Modelo Logístico</h2>

In [35]:
from sklearn.linear_model import LogisticRegression
## inicializando o modelo
logit = LogisticRegression()
## Ajustando o modelo com valores X e Y do conjunto de dados
logit.fit(train_X , train_y)

LogisticRegression()

In [37]:
from sklearn.model_selection import cross_val_predict
y_trained_pred = cross_val_predict(logit,train_X,train_y)

In [38]:
y_trained_prob = cross_val_predict(logit,train_X,train_y,method="predict_proba")

In [39]:
## Importando as metricas
from sklearn import metrics
## Definindo a matriz para desenhar a matriz de confusão do real e
## rótulos de classe previstos
def draw_cm( actual, predicted ):
    # Invocando confusão_matrix do pacote métrico. O Matrix
    # será orientado como [1,0], ou seja, as classes com rótulo 1 serão
    # representado pela primeira linha e 0 como segunda linha
    cm= metrics.confusion_matrix(actual,predicted)
    # A confusão será plotada como mapa de calor para melhor visualização
    # Os rótulos são configurados para melhor interpretação da trama
    sn.heatmap(cm, annot=True, fmt='.2f',xticklabels = ['No Failure', 'Failure'], yticklabels = ['No Failure', 'Failure'] )
    plt.ylabel('True label') 
    plt.xlabel('Predicted label') 
    plt.show()

In [40]:
draw_cm(train_y , y_trained_pred)

In [41]:
print(metrics.classification_report(train_y , y_trained_pred))

              precision    recall  f1-score   support

           0       0.91      0.97      0.94      7717
           1       0.77      0.55      0.64      1604

    accuracy                           0.89      9321
   macro avg       0.84      0.76      0.79      9321
weighted avg       0.89      0.89      0.89      9321



In [42]:
y_trained_score = y_trained_prob[:,1]
fpr,tpr,threshold = metrics.roc_curve(train_y,y_trained_score)

In [43]:
def plot_roc_curve(fpr,tpr):
    plt.plot(fpr,tpr,linewidth=2)
    plt.plot([0,1],[0,1],"k--")
    plt.xlabel('False Positive Rate or [1 - True Negative Rate]')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.grid(True)
    plt.figure(figsize=(8, 6))
    plt.show()

In [45]:
plot_roc_curve(fpr,tpr)

In [46]:
metrics.roc_auc_score(train_y,y_trained_score)

0.8874980328109363

<h2>Construindo Modelo de Árvore de Decisão</h2>

In [47]:
from sklearn.tree import DecisionTreeClassifier
dec_tree = DecisionTreeClassifier()
dec_tree.fit(train_X , train_y)

DecisionTreeClassifier()

In [48]:
from sklearn.model_selection import cross_val_predict
tree_pred = cross_val_predict(dec_tree,train_X,train_y)
tree_prob = cross_val_predict(dec_tree,train_X,train_y,method="predict_proba")
tree_score = tree_prob[:,1]

In [49]:
from sklearn.metrics import precision_score , recall_score , confusion_matrix

In [50]:
draw_cm(train_y,tree_pred)

In [51]:
tree_fpr , tree_tpr , thrshold = metrics.roc_curve(train_y,tree_score)

In [52]:
plot_roc_curve(tree_fpr,tree_tpr)

In [53]:
metrics.roc_auc_score(train_y,tree_score)

0.9877996711603135

<h2>Construindo Modelo Florestal Aleatório</h2>

In [55]:
from sklearn.ensemble import RandomForestClassifier
random_clf = RandomForestClassifier(max_depth=25,n_estimators=40)

In [56]:
random_clf.fit(train_X,train_y)

RandomForestClassifier(max_depth=25, n_estimators=40)

In [57]:
from sklearn.model_selection import cross_val_predict,cross_val_score
random_clf_pred = cross_val_predict(random_clf,train_X,train_y)
random_clf_prob = cross_val_predict(random_clf,train_X,train_y,method="predict_proba")
random_clf_score = random_clf_prob[:,1]

In [58]:
from sklearn.metrics import precision_score,recall_score,precision_recall_curve
precision,recall,threshold = precision_recall_curve(train_y,random_clf_score)

In [59]:
def precision_recall_curve_VS_threshold(Precision,recall,threshold):
    plt.plot(threshold,precision[:-1],"b--",label="precision")
    plt.plot(threshold,recall[:-1],"g--",label="recall")
    plt.xlabel('Threshold')
    plt.ylabel('Score')
    plt.title('precision_recall_curve VS threshold')
    plt.legend(loc='lower right') 
    plt.grid(True)
    plt.figure(figsize=(8, 6))
    plt.show()

In [60]:
precision_recall_curve_VS_threshold(precision,recall,threshold)

In [61]:
draw_cm(train_y,random_clf_pred)

In [63]:
feature_rank = pd.DataFrame( { 'feature': encodeded_df.columns,'importance': random_clf.feature_importances_ } )
feature_rank[feature_rank["importance"]>0.01].sort_values('importance', ascending = False)
# feature_rank.head(10)
# feature_rank[feature_rank["importance"]>0.05]
# plt.figure(figsize=(18, 16))
# sn.barplot( y = 'feature', x =features_ ,data = feature_rank );

,feature,importance
4,Torque [Nm],0.288669
3,Rotational speed [rpm],0.251481
5,Tool wear [min],0.173132
0,UDI,0.102962
1,Air temperature [K],0.102318
2,Process temperature [K],0.066675


In [64]:
feature_rank['cumsum'] = feature_rank.importance.cumsum() * 100 
feature_rank.iloc[: , :]

,feature,importance,cumsum
0,UDI,0.102962,10.296175
1,Air temperature [K],0.102318,20.527977
2,Process temperature [K],0.066675,27.195441
3,Rotational speed [rpm],0.251481,52.343564
4,Torque [Nm],0.288669,81.210496
5,Tool wear [min],0.173132,98.523647
6,Type_L,0.007904,99.314095
7,Type_M,0.006859,100.000000


In [65]:
from sklearn.model_selection import GridSearchCV
# Define a grade de hiperparâmetros para busca usando GridSearchCV.
# Aqui, 'max_depth' representa a profundidade máxima de cada árvore,
# e 'n_estimators' é o número total de árvores na floresta.
tuned_parameters = [{'max_depth': [5,10, 15,20,25,30],'n_estimators': [25,30,35,40,45,50]}]
## Configurando a pesquisa com os parâmetros ajustáveis
clf = GridSearchCV(random_clf,tuned_parameters, cv=5, scoring='roc_auc')
## Ajustando o conjunto de treinamento
clf.fit(train_X, train_y)

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(max_depth=25, n_estimators=40),
             param_grid=[{'max_depth': [5, 10, 15, 20, 25, 30],
                          'n_estimators': [25, 30, 35, 40, 45, 50]}],
             scoring='roc_auc')

In [66]:
clf.best_score_

0.9997042011155057

In [67]:
clf.best_params_

{'max_depth': 30, 'n_estimators': 45}

In [ ]:
## Modelo de teste em dados de teste

<h2>Testando</h2>

Como estamos testando nosso modelo no conjunto de teste, não queremos ajustar o modelo ao conjunto de teste, então apenas usamos o método de previsão no melhor parâmetro e no melhor algoritmo
<p>melhor algoritmo: floresta aleatória</p>

In [68]:
pred = random_clf.predict(test_X)

In [69]:
draw_cm(test_y,pred)